# VR CO2 Study - Data processing

This notebook does the following:

1. x
2. y
3. z

Input: input
Output: output

In [2]:
# Imports
import json
import os
import pandas as pd
from utils.enums import AirFiles, CO2Files
from utils.load_data import load_data_with_event_matching

In [3]:
# Open file index JSON for reading
file_index = pd.read_json(os.path.join(os.getcwd(), 'temp/file_index.json'))
file_index = file_index.sort_index()

In [4]:
# Example how to retrieve file names for a participant
participant_to_retrieve = 1
air_mask_data = file_index[0][participant_to_retrieve-1][AirFiles.MASK.value]
air_event_data = file_index[0][participant_to_retrieve-1][AirFiles.EVENT.value]

In [5]:
participant_df = load_data_with_event_matching(air_mask_data, True, air_event_data)

D:\co2-study\utils\load_data.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Event'][event_row_index] = events[i]


In [6]:
participant_df

,Frame#,Time,Faceplate/FaceState,Faceplate/FitState,Emg/ContactStates[RightOrbicularis],Emg/Contact[RightOrbicularis],Emg/Raw[RightOrbicularis],Emg/RawLift[RightOrbicularis],Emg/Filtered[RightOrbicularis],Emg/Amplitude[RightOrbicularis],Emg/ContactStates[RightZygomaticus],Emg/Contact[RightZygomaticus],Emg/Raw[RightZygomaticus],Emg/RawLift[RightZygomaticus],Emg/Filtered[RightZygomaticus],Emg/Amplitude[RightZygomaticus],Emg/ContactStates[RightFrontalis],Emg/Contact[RightFrontalis],Emg/Raw[RightFrontalis],Emg/RawLift[RightFrontalis],Emg/Filtered[RightFrontalis],Emg/Amplitude[RightFrontalis],Emg/ContactStates[CenterCorrugator],Emg/Contact[CenterCorrugator],Emg/Raw[CenterCorrugator],Emg/RawLift[CenterCorrugator],Emg/Filtered[CenterCorrugator],Emg/Amplitude[CenterCorrugator],Emg/ContactStates[LeftFrontalis],Emg/Contact[LeftFrontalis],Emg/Raw[LeftFrontalis],Emg/RawLift[LeftFrontalis],Emg/Filtered[LeftFrontalis],Emg/Amplitude[LeftFrontalis],Emg/ContactStates[LeftZygomaticus],Emg/Contact[LeftZygomaticus],Emg/Raw[LeftZygomaticus],Emg/RawLift[LeftZygomaticus],Emg/Filtered[LeftZygomaticus],Emg/Amplitude[LeftZygomaticus],Emg/ContactStates[LeftOrbicularis],Emg/Contact[LeftOrbicularis],Emg/Raw[LeftOrbicularis],Emg/RawLift[LeftOrbicularis],Emg/Filtered[LeftOrbicularis],Emg/Amplitude[LeftOrbicularis],HeartRate/Average,Ppg/Raw.ppg,Ppg/Raw.proximity,Accelerometer/Raw.x,Accelerometer/Raw.y,Accelerometer/Raw.z,Magnetometer/Raw.x,Magnetometer/Raw.y,Magnetometer/Raw.z,Gyroscope/Raw.x,Gyroscope/Raw.y,Gyroscope/Raw.z,Pressure/Raw,unix_timestamp,Event
0,1,1.651082e+09,0,0,17,0.0,0.002248,0,0.012367,0.000000,0,0.0,-0.007862,0,-8.967082e-03,0.000000,0,0.0,-0.006634,0,0.000545,0.000000,0,0.0,-0.028150,0,-3.951033e-03,0.000000,0,0.0,-0.005717,0,-1.136740e-03,0.000000,0,0.0,0.016474,0,-3.594438e-03,0.000000,0,0.0,0.052545,0,-0.016730,0.000000,0.00,0,0,0.996147,-0.057392,-0.166374,-62.905585,1.958492,20.783397,1.068148,-1.434370,0.762963,0,1.651082e+09,
1,2,1.651082e+09,0,0,0,0.0,0.000545,0,0.005631,0.000000,0,0.0,-0.009789,0,-3.812472e-03,0.000000,0,0.0,-0.008044,0,0.000575,0.000000,0,0.0,-0.029742,0,-1.746138e-03,0.000000,0,0.0,-0.007278,0,-2.049208e-04,0.000000,0,0.0,0.014643,0,-1.377781e-03,0.000000,0,0.0,0.051726,0,-0.006996,0.000000,0.00,0,0,0.996147,-0.057392,-0.166374,-62.905585,1.958492,20.783397,1.068148,-1.434370,0.762963,0,1.651082e+09,
2,3,1.651082e+09,0,0,0,0.0,0.002259,0,0.000933,0.000000,0,0.0,-0.007820,0,-9.137392e-04,0.000000,0,0.0,-0.006582,0,-0.000197,0.000000,0,0.0,-0.028137,0,-3.163020e-04,0.000000,0,0.0,-0.005687,0,-3.031890e-04,0.000000,0,0.0,0.016468,0,-5.187591e-04,0.000000,0,0.0,0.052960,0,-0.001759,0.000000,0.00,0,0,0.996147,-0.057392,-0.166374,-62.905585,1.958492,20.783397,1.068148,-1.434370,0.762963,0,1.651082e+09,
3,4,1.651082e+09,0,0,0,0.0,0.000562,0,-0.002309,0.000000,0,0.0,-0.009830,0,1.792034e-03,0.000000,17,0.0,-0.008124,0,0.000038,0.000000,0,0.0,-0.029747,0,7.769267e-04,0.000000,0,0.0,-0.007283,0,3.628333e-04,0.000000,0,0.0,0.014596,0,7.654428e-04,0.000000,0,0.0,0.051616,0,0.003665,0.000000,0.00,0,0,0.996147,-0.057392,-0.166374,-62.905585,1.958492,20.783397,1.068148,-1.434370,0.762963,0,1.651082e+09,
4,5,1.651082e+09,0,0,0,0.0,0.002277,0,-0.002642,0.000000,0,0.0,-0.007933,0,1.657367e-03,0.000000,17,0.0,-0.006686,0,-0.000412,0.000000,0,0.0,-0.028149,0,8.174181e-04,0.000000,0,0.0,-0.005691,0,-7.510185e-06,0.000000,0,0.0,0.016448,0,4.602671e-04,0.000000,0,0.0,0.052774,0,0.003105,0.000000,0.00,0,0,0.996147,-0.057392,-0.166374,-62.905585,1.958492,20.783397,1.068148,-1.434370,0.762963,0,1.651082e+09,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426804,1426805,1.651083e+09,1,9,255,13514.0,-0.033371,0,0.000001,0.000005,255,21070.0,-0.004965,0,-1.192093e-07,0.000025,255,11478.0,-0.005722,0,-0.000025,0.000025,255,7762.0,-0.021287,0